# Setup Environment, Load Data

In [ ]:
import smtplib
from email.mime.text import MIMEText

# Email configuration
SMTP_SERVER = "smtp.gmail.com"  # For Gmail, change if using Outlook or another service
SMTP_PORT = 465  # SSL port (or use 587 for TLS)
SENDER_EMAIL = "abhaymathur1000@gmail.com"  # Replace with your email
SENDER_PASSWORD = "blom axpb quot zcdv"  # Replace with your email password
RECEIVER_EMAIL = "abhaymathur1000@gmail.com"  # Your email (or another recipient)

def send_email(subject, body=""):
    """Sends an email notification"""
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = RECEIVER_EMAIL

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")



# Notify when the script completes
# send_email("✅ Script Completed", "Your Python script has finished running.")


In [ ]:
#Import Libraries
import os
import json
import csv
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import re
import time
import spacy
import kagglehub


from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from IPython.display import display, clear_output

import pyarrow
import fastparquet
import dask.dataframe as dd

#NLP Packages
# import tensorflow as tf
# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
# from tensorflow.keras.preprocessing.text import one_hot
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Activation, Dropout, Dense
# from tensorflow.keras.layers import Flatten, LSTM
# from tensorflow.keras.layers import GlobalMaxPooling1D
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Embedding
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.layers import Input
# from tensorflow.keras.layers import Concatenate
# import transformers
# from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

# Save data

In [ ]:
def save_df(df, filename, path, num_files):
    save_folder = os.path.join(path, filename)

    num_folder = 0
    while os.path.exists(save_folder):
        num_folder += 1
        save_folder = os.path.join(path, filename + f"_{num_folder}")
    
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    chunk_size = len(df) // num_files

    for i in range(num_files):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i != num_files - 1 else len(df)  # Ensure last chunk gets all remaining rows
        df_chunk = df.iloc[start_idx:end_idx]
        df_chunk.to_parquet(os.path.join(save_folder, f"output_part_{i}.parquet"), engine="pyarrow", index=False)

    print("Output folder", save_folder)
    return save_folder

In [ ]:
def save_dask_df(df, filename, path, num_files):
    save_folder = os.path.join(path, filename)

    num_folder = 0
    while os.path.exists(save_folder):
        num_folder += 1
        save_folder = os.path.join(path, filename + f"_{num_folder}")
    
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Repartition the DataFrame to the desired number of partitions
    df = df.repartition(npartitions=num_files)

    # Save the DataFrame to Parquet files
    df.to_parquet(os.path.join(save_folder, "*.parquet"), engine="pyarrow", write_index=False)

    print("Output folder", save_folder)
    return save_folder

# Read in data

In [ ]:
def load_dask_data(save_folder):
    # Load all Parquet files as a Dask DataFrame (lazy loading)
    print("Loading from folder: ", save_folder)
    df_dask = dd.read_parquet(os.path.join(save_folder, "*.parquet"), engine="pyarrow")

    display(df_dask.npartitions)

    display(df_dask.head(5))

    # for partition in df_dask.to_delayed():
        # partition_df = partition.compute()  # Load only one partition into memory
    #     print(partition_df.shape)  # Process chunk-by-chunk

    return df_dask

In [ ]:
if os.path.exists("/kaggle"):
    path = "/kaggle/input/yelp-dataset"
elif os.path.exists('/home/abhaydesktop/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4'):
    path = '/home/abhaydesktop/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4'
else:
    path = kagglehub.dataset_download("yelp-dataset/yelp-dataset")
print("Path to dataset files:", path)



def load_data(path, filename, chunk_size=10000):
    chunks = pd.read_json(os.path.join(path, filename), lines=True, chunksize=chunk_size)

    df_list = []

    for chunk in chunks:
        df_list.append(chunk)


    return pd.concat(df_list, ignore_index=True)



print("\nDataset Load Times:\n")
start_time = time.time()
businesses_df = load_data(path, "yelp_academic_dataset_business.json")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Business Load Time: {elapsed_time:.4f} seconds")

# start_time = time.time()
# tips_df = load_data(path, "yelp_academic_dataset_tip.json")
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Tips Load Time: {elapsed_time:.4f} seconds")

start_time = time.time()
reviews_df = load_data(path, "yelp_academic_dataset_review.json")
# num_rev_load = 2000000
num_rev_load = 100000000
if num_rev_load < len(reviews_df):
    reviews_df = reviews_df.sample(n=num_rev_load)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Review Load Time: {elapsed_time:.4f} seconds")




# Clean Data

Current Thoughts:
* Should we be counting categories weighted based on reviews, or should we just count categories by businesses?
* Do we need to lemmatize for a BERT model considering it has been trained? Also, can lemmatized words be tokenized?

In [ ]:
#Filter out Data that Belongs to Businesses Under a Certain Threshold
minimum_business_reviews = 30
reviews_df = reviews_df[reviews_df['business_id'].map(reviews_df['business_id'].value_counts()).gt(minimum_business_reviews)]

#Create Merged DataFrame of Remaining Reviewed Businesses, and Split Categories into List
df_rb = pd.merge(reviews_df, businesses_df, on="business_id")
df_rb['categories'] = df_rb['categories'].str.split(", ")

#Print Metrics
print(f"\nNumber of Total Businesses Represented: {len(df_rb['business_id'].unique())}")
print(f"Number of Total Reviews: {len(df_rb)}")
print(f"Percentage of Reviews Kept: {len(df_rb)/num_rev_load*100}%")

In [ ]:
#Dictionary to Store Categories, and Variables to Report Later
all_unique_cats = {}
total_cats_before = df_rb['categories'].apply(len).sum()
len_before = len(df_rb)

#Iterate through each Business' Categories and Count Occurances (Weighted Based on Reviews)
for cats in df_rb['categories']:
    for cat in cats:
        if cat in all_unique_cats:
            all_unique_cats[cat] += 1
        else:
            all_unique_cats[cat] = 1

#Number of Top Categories to Keep
num_cats = 30
sorted_cats = sorted(all_unique_cats.items(), key=lambda x: x[1], reverse=True)

#Select Top Categories
top_cats = [x[0] for x in sorted_cats[:num_cats]]

#Remove Reviews that Belong to Businesses with None of the Top Categories
df_rb = df_rb[df_rb['categories'].apply(lambda x: bool(set(x) & set(top_cats)))]
# df_rb = df_rb[df_rb['categories'].apply(lambda x: any(cat in top_cats for cat in x) if x else False)]

#Remove Categories from Remaining Reviews that are not in the Top Categories
# df_rb['categories'] = df_rb['categories'].apply(lambda x: ([i for i in x if i in top_cats]))
top_cats_set = set(top_cats)
df_rb['categories'] = df_rb['categories'].apply(lambda x: [i for i in x if i in top_cats_set])

#Count Total Number of Categories
# total_cats_after = df_rb['categories'].apply(len).sum()
total_cats_after = sum(len(cats) for cats in df_rb['categories'])

print(f"\nTotal Initial Unique Categories: {len(all_unique_cats)}")
print(f"Number of Total Reviews: {len(df_rb)}")
print(f"Percentage of Reviews Kept: {len(df_rb)/num_rev_load*100}%")
print(f"Average Number of Categories Before: {total_cats_before/len_before}")
print(f"Average Number of Categories After: {total_cats_after/len(df_rb)}")

In [ ]:
tmp_save_path = save_df(df_rb, 'tmp_df_rb', 'data', 20)

df_rb_dask = load_dask_data(tmp_save_path)

send_email("At the leave off point", "")

In [ ]:
# # Load the spaCy English model

os.system("./venv/bin/python -m spacy download en_core_web_sm")
# os.system("python -m spacy download en_core_web_md")

nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])


def lemmatize(text):
    # Process the text using spaCy
    # start_time = time.time()
    doc = nlp(text)
    # end_time = time.time()
    # elapsed_time = end_time - start_time
    # print(f"Step 1 Load Time: {elapsed_time:.4f} seconds")
    
    # Extract lemmatized tokens
    # start_time = time.time()
    lemmatized_tokens = [token.lemma_ for token in doc]
    # end_time = time.time()
    # elapsed_time = end_time - start_time
    # print(f"Step 2 Load Time: {elapsed_time:.4f} seconds")
    
    # Join the lemmatized tokens into a sentence
    lemmatized_text = ' '.join(lemmatized_tokens)

    return lemmatized_text



In [ ]:
#Clean review text field
import nltk
nltk.download('stopwords')
sw = stopwords.words('english')
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z?.,!¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    html = re.compile(r'<.*?>')
    text = html.sub(r'',text) #Removing html tags
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^,' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = " ".join(text) #removing stopwords
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    return text

# df_rb['text'] = df_rb['text'].apply(lambda x: clean_text(x))
partitions = []
for partition in df_rb_dask.to_delayed():
    partition_df = partition.compute()  # Load only one partition into memory
    partition_df['text'] = partition_df['text'].apply(lambda x: clean_text(x))
    partition_df['text'] = partition_df['text'].apply(lambda x: lemmatize(x))
    partitions.append(partition_df)
df_rb_dask = dd.from_pandas(pd.concat(partitions), npartitions=len(partitions))

In [ ]:
save_df(pd.concat(partitions), 'new_reviews_parquet', 'data', 20)

In [ ]:
send_email("Download completed at home", "")